# Import Hook

In [1]:
%load_ext literary.notebook

In [2]:
import os
import sys
import typing as tp
from pathlib import Path

from nbconvert import Exporter
from traitlets import Bool, Instance, Type

from ..core.exporter import LiteraryExporter
from ..core.project import ProjectOperator
from .finder import inject_loaders
from .loader import NotebookLoader
from .patch import patch

Here we implement a `ProjectImporter` class. Because it operators upon the current project, it is a `ProjectOperator` subclass. We define a few useful configuration traits, such as the exporter class and the output directory.

In [3]:
class ProjectImporter(ProjectOperator):
    exporter = Instance(Exporter)
    exporter_class = Type(LiteraryExporter, help="Exporter class").tag(config=True)
    set_except_hook = Bool(
        help="overwrite `sys.excepthook` to correctly display tracebacks"
    ).tag(config=True)

In [10]:
@patch(ProjectImporter)
@default("_exporter")
def _exporter_default(self):
    return self.exporter_class(parent=self)

AttributeError: 'DefaultHandler' object has no attribute '__name__'

In [4]:
@patch(ProjectImporter)
def determine_package_name(self, path: Path) -> str:
    """Determine the corresponding importable name for a package directory given by
    a particular file path. Return `None` if path is not contained within `sys.path`.

    :param path: path to package
    :return:
    """
    for p in sys.path:
        # If the cwd is only on `sys.path` exactly, then it can't
        # be a submodule. However, let's first look for parent paths
        if str(path) == p:
            continue

        # Resolve path relative to `sys.path`
        try:
            relative_path = path.relative_to(p)
        except ValueError:
            continue

        return ".".join(relative_path.parts)

    # Nothing was found on `sys.path`, so we abort
    return None

In [5]:
@patch(ProjectImporter)
def install(self, ipython):
    """Install notebook import hook

    Don't allow the user to specify a custom search path, because we also need this to
    interoperate with the default Python module importers which use sys.path

    :return:
    """

    # Make notebook packages importable by adding package root path to sys.path
    sys.path.append(str(self.packages_path))

    # Create the exporter
    exporter = self.exporter_class(parent=self)

    # Create notebook loader factory
    def create_notebook_loader(fullname, path):
        return NotebookLoader(fullname, path, exporter=exporter)

    # Inject notebook loader into path_hooks
    inject_loaders(sys.path_hooks, (create_notebook_loader, [".ipynb"]))

    # Python's C-level traceback reporting doesn't call `linecache`, and so retrieves
    # the underlying notebook source instead of the generated Python code
    if self.set_except_hook:
        sys.excepthook = traceback.print_exception

    # Update user namespace
    ipython.user_ns.update(
        {
            "__package__": self.determine_package_name(Path.cwd()),
            "patch": patch,
        }
    )